# Merkle Tree

In [1]:
import bitcoin #pybitcointools
from bitcoin import sha256
import binascii

In [2]:
class Merkle:
    '''
    Merkle tree
    
    Used for storing transactions in the blockchain
    '''
    def __init__(self, txs):
        txs_hashes = [sha256(binascii.unhexlify(tx)) for tx in txs]
        self.levels = list()
        self._build(txs_hashes)
        
    def _build(self, leaves):
        if len(leaves) == 1:
            self.levels.append(leaves)
            return None
        elif len(leaves) % 2 == 1:
            leaves.append(leaves[-1])
        self.levels.append(leaves)
        new_level = list()
        for i in range(len(leaves)//2):
            new_level.append(sha256(binascii.unhexlify(leaves[i*2] + leaves[i*2 + 1])))
        self._build(new_level)
        
    def show(self):
        for level in reversed(self.levels):
            print(level)
            
    def get_root(self):
        return self.levels[-1][0]
    
    def get_path(self, tx_hash):
        idx = self.levels[0].index(tx_hash) 
        path = list()
        for i in range(len(self.levels)-1):
            if idx % 2 == 0:
                path_idx = idx + 1
            else:
                path_idx = idx - 1
            path.append(self.levels[i][path_idx])
            idx = idx // 2
        return path
        
    def path_verify(self, tx_hash, path):
        res = tx_hash
        for node in path:
            res = sha256(binascii.unhexlify(res + node))
        return res == self.get_root()

In [3]:
txs = ['0000000000000000000000000000000000000000000000000000000000000000',
       '0000000000000000000000000000000000000000000000000000000000000011',
       '0000000000000000000000000000000000000000000000000000000000000022']
tree = Merkle(txs)

In [4]:
tree.show()

['34ca81403f7450b6c82dc28d54dc495f4f024d0b46e4cf7d687c9530941b1ee7']
['a8a6080ff7c50ce21c2d367239dce9e9612fa0da51a0a2173c1430a36fe35ad9', 'd7643aef6e9059376a8b6647230aaf30a7fd873da89a83f96bbf93e17b27354f']
['66687aadf862bd776c8fc18b8e9f8e20089714856ee233b3902a591d0d5f2925', '99fdc3a44c06c65a307ea38acda009243287ccbbdb2b0ce423a25bb9b525d7f2', '46f3ce0180a8791e8c622020c2bbdf688405a3e3cf9a8061309dc71fcaa4b7ac', '46f3ce0180a8791e8c622020c2bbdf688405a3e3cf9a8061309dc71fcaa4b7ac']


In [5]:
path = tree.get_path('66687aadf862bd776c8fc18b8e9f8e20089714856ee233b3902a591d0d5f2925')
print(path)

['99fdc3a44c06c65a307ea38acda009243287ccbbdb2b0ce423a25bb9b525d7f2', 'd7643aef6e9059376a8b6647230aaf30a7fd873da89a83f96bbf93e17b27354f']


In [6]:
tree.path_verify('66687aadf862bd776c8fc18b8e9f8e20089714856ee233b3902a591d0d5f2925', path)

True